# 自転車犯罪マップ

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/crime map.png" width=600>

このラボの目的:

- オープンデータソースからデータを取得し、Python ノートブックにインポート
- 複数の列から単一の住所フィールドを作成
- 緯度と経度の座標を割り当てて各行をジオコーディング
- 美しいチャートや地図を作成

## ライブラリをインポートする

このラボで使うライブラリーを一気にインポートしよう。

In [1]:
## for spatial analysis
import geopandas as gpd

## for data analysis
import pandas as pd

## for pretty charts
import plotly.express as px

# for plotly themes
import plotly.io as pio

## for URL requests
import urllib.request
import requests

## for maps
import folium
from folium import plugins

# オープンデータとは？

どんなプロジェクトでも、どんなマップでもデータが必要です。でも「いい」データって意外となかったりするので、イメージしていたマップが作れないことがよくある。そこで、最近政府機関がオープンデータを提供する方向性があり、あらゆる行政がCSVやEXCELフォーマットでデータをダウンロードできるように提供している。

Let's find some open data!

Here is an example:

https://www.pref.chiba.lg.jp/shoufuku/opendata/techoutoukei.html

## 自転車盗難データをダウンロード

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/Chiba%20police.png" width=400>

まずは千葉県警察のサイトにアクセス。そこから次に手順でデータをダウンロード：

➡️ https://www.police.pref.chiba.jp/

➡️ 安全な暮らし

➡️ 地域の防犯

➡️ あなたの町の犯罪情勢

➡️ オープンデータ

➡️ 自転車盗（CSV）

ダウンロードしたファイルを `chibike.csv` と名付けて、このフォルダー（Week08) にセーブ。

In [22]:
# ダウンロードしたデータを読み込もう
df = pd.read_csv('chicar.csv', encoding='cp932')

In [23]:
# データの情報
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             627 non-null    object
 1   手口             627 non-null    object
 2   管轄警察署（発生地）     627 non-null    object
 3   管轄交番・駐在所（発生地）  627 non-null    object
 4   市区町村コード（発生地）   627 non-null    int64 
 5   都道府県（発生地）      627 non-null    object
 6   市区町村（発生地）      627 non-null    object
 7   町丁目（発生地）       625 non-null    object
 8   発生年月日（始期）      627 non-null    object
 9   発生時（始期）        627 non-null    object
 10  発生場所           627 non-null    object
 11  発生場所の詳細        627 non-null    object
 12  施錠関係           627 non-null    object
 13  盗難防止装置の有無      627 non-null    object
 14  現金以外の主な被害品     627 non-null    object
dtypes: int64(1), object(14)
memory usage: 73.6+ KB


Wow! ７千件もある！ちょっと多いので場所で絞りましょう。<h1>「柏」</h1>だけのデータフレームを作ろう。

In [24]:
df

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,施錠関係,盗難防止装置の有無,現金以外の主な被害品
0,窃盗,自動車盗,千葉中央,千葉寺駅前交番,121011,千葉県,千葉市中央区,青葉町,2022/4/9,18,一戸建住宅,駐車（輪）場,キーなし,なし,なし
1,窃盗,自動車盗,千葉中央,千葉寺駅前交番,121011,千葉県,千葉市中央区,千葉寺町,2022/3/24,23,一戸建住宅,駐車（輪）場,キーなし,あり,乗用自動車
2,窃盗,自動車盗,千葉中央,千葉駅前交番,121011,千葉県,千葉市中央区,新町,2022/5/23,10,道路上,その他,キーなし,なし,乗用自動車
3,窃盗,自動車盗,千葉中央,千葉駅前交番,121011,千葉県,千葉市中央区,千葉寺町,2022/4/25,16,駐車（輪）場,駐車（輪）場,キーなし,あり,乗用自動車
4,窃盗,自動車盗,千葉中央,星久喜交番,121011,千葉県,千葉市中央区,仁戸名町,2022/7/27,1,その他の住宅（３階建て以下共同住宅等）,駐車（輪）場,キーなし,なし,乗用自動車
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,窃盗,自動車盗,四街道,大日交番,122289,千葉県,四街道市,大日,2022/11/4,0,その他,駐車（輪）場,キーなし,なし,乗用自動車
623,窃盗,自動車盗,四街道,大日交番,122289,千葉県,四街道市,鹿放ケ丘,2022/7/19,17,その他,駐車（輪）場,キーなし,なし,乗用自動車
624,窃盗,自動車盗,四街道,大日交番,122289,千葉県,四街道市,鹿放ケ丘,2022/11/28,17,その他,その他,キーなし,なし,貨物自動車
625,窃盗,自動車盗,四街道,旭交番,122289,千葉県,四街道市,小名木,2022/2/13,23,一戸建住宅,駐車（輪）場,キーあり,あり,乗用自動車


In [25]:
kashiwa = df[df['管轄警察署（発生地）'] == '柏'].copy()

In [26]:
# データをチェック
kashiwa.sample(5)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,施錠関係,盗難防止装置の有無,現金以外の主な被害品
140,窃盗,自動車盗,柏,北柏駅前交番,122173,千葉県,柏市,北柏４丁目,2022/4/15,18,その他の住宅（３階建て以下共同住宅等）,駐車（輪）場,キーなし,なし,乗用自動車
175,窃盗,自動車盗,柏,船戸駐在所,122173,千葉県,柏市,船戸３丁目,2022/7/24,20,一戸建住宅,その他,キーなし,あり,乗用自動車
185,窃盗,自動車盗,柏,藤ヶ谷駐在所,122173,千葉県,柏市,藤ケ谷,2022/4/29,19,その他,その他,キーあり,あり,乗用自動車
176,窃盗,自動車盗,柏,船戸駐在所,122173,千葉県,柏市,船戸３丁目,2022/7/29,2,一戸建住宅,駐車（輪）場,キーなし,あり,なし
151,窃盗,自動車盗,柏,大津ヶ丘交番,122173,千葉県,柏市,塚崎,2022/4/4,22,その他,駐車（輪）場,キーあり,なし,なし


In [27]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 132 to 194
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             63 non-null     object
 1   手口             63 non-null     object
 2   管轄警察署（発生地）     63 non-null     object
 3   管轄交番・駐在所（発生地）  63 non-null     object
 4   市区町村コード（発生地）   63 non-null     int64 
 5   都道府県（発生地）      63 non-null     object
 6   市区町村（発生地）      63 non-null     object
 7   町丁目（発生地）       63 non-null     object
 8   発生年月日（始期）      63 non-null     object
 9   発生時（始期）        63 non-null     object
 10  発生場所           63 non-null     object
 11  発生場所の詳細        63 non-null     object
 12  施錠関係           63 non-null     object
 13  盗難防止装置の有無      63 non-null     object
 14  現金以外の主な被害品     63 non-null     object
dtypes: int64(1), object(14)
memory usage: 7.9+ KB


### `value_counts`でチャート

このデータを見て、咄嗟に知りたいものってなんでしょう？

例えば、<h1>「1日の何時に自転車の盗難が一番発生するの？」</h1>の質問に答えるためにはどのようなデータ分析が必要でしょうか？

では、そのチャートを作ってみましょう。

データには **【発生時（始期）】** のカラムがあるので、各時間帯のカウントを`value_counts()`で調べる。最後に足される`reset_index()`で結果をデータフレームに変換します。

In [28]:
# create a new variable with hourly counts
time = kashiwa['発生時（始期）'].value_counts().reset_index()

time

,発生時（始期）,count
0,18,9
1,21,7
2,22,7
3,20,7
4,19,6
5,17,4
6,13,2
7,0,2
8,2,2
9,1,2


この結果は良いが、カラム名（ヘッダー）を直す必要がある。

In [29]:
# fix headers
time.columns = ['発生時（始期）','件数']
time

,発生時（始期）,件数
0,18,9
1,21,7
2,22,7
3,20,7
4,19,6
5,17,4
6,13,2
7,0,2
8,2,2
9,1,2


いよいよ準備万端。この新しいデータフレームでチャートを作ろう。この場合は plotly express の bar charts を参考。

https://plotly.com/python/bar-charts/

In [30]:
fig = px.bar(time,x='発生時（始期）',y='件数')
fig.show()

ありゃ？順番が件数の多い順になってる。実はx軸のオプションっていっぱいあるんだ！その中の一つでカテゴリーの順番を設定できる。

https://plotly.com/python/categorical-axes/

In [31]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数'
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

チャートのテンプレートを変えることでルックスが変わる。

`template`のオプションはこちらから：

```["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white"]```

一つずつ試してみよう。

In [33]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数',
            template='ggplot2' # change this to see other styles
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

### Make your own charts

では、ここで他のチャートを作ってみよう。

例：被害者の年齢、被害者の職業

In [34]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 132 to 194
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             63 non-null     object
 1   手口             63 non-null     object
 2   管轄警察署（発生地）     63 non-null     object
 3   管轄交番・駐在所（発生地）  63 non-null     object
 4   市区町村コード（発生地）   63 non-null     int64 
 5   都道府県（発生地）      63 non-null     object
 6   市区町村（発生地）      63 non-null     object
 7   町丁目（発生地）       63 non-null     object
 8   発生年月日（始期）      63 non-null     object
 9   発生時（始期）        63 non-null     object
 10  発生場所           63 non-null     object
 11  発生場所の詳細        63 non-null     object
 12  施錠関係           63 non-null     object
 13  盗難防止装置の有無      63 non-null     object
 14  現金以外の主な被害品     63 non-null     object
dtypes: int64(1), object(14)
memory usage: 7.9+ KB


In [46]:
kashiwa['発生場所'].value_counts()

発生場所
駐車（輪）場                 21
一戸建住宅                  16
その他                    13
その他の住宅（３階建て以下共同住宅等）    10
４階建て以上共同住宅              2
道路上                     1
Name: count, dtype: int64

In [47]:
# create a new variable with hourly counts
place = kashiwa['発生場所'].value_counts().reset_index()

place

,発生場所,count
0,駐車（輪）場,21
1,一戸建住宅,16
2,その他,13
3,その他の住宅（３階建て以下共同住宅等）,10
4,４階建て以上共同住宅,2
5,道路上,1


In [48]:
# fix headers
place.columns = ['発生場所','件数']
place

,発生場所,件数
0,駐車（輪）場,21
1,一戸建住宅,16
2,その他,13
3,その他の住宅（３階建て以下共同住宅等）,10
4,４階建て以上共同住宅,2
5,道路上,1


In [39]:
fig = px.bar(place,x='発生場所',y='件数')
fig.show()

In [49]:
kashiwa['現金以外の主な被害品'].value_counts()

現金以外の主な被害品
乗用自動車    55
なし        5
特殊自動車     2
貨物自動車     1
Name: count, dtype: int64

In [50]:
# create a new variable with hourly counts
goods = kashiwa['現金以外の主な被害品'].value_counts().reset_index()

goods

,現金以外の主な被害品,count
0,乗用自動車,55
1,なし,5
2,特殊自動車,2
3,貨物自動車,1


In [51]:
# fix headers
goods.columns = ['現金以外の主な被害品','件数']
goods

,現金以外の主な被害品,件数
0,乗用自動車,55
1,なし,5
2,特殊自動車,2
3,貨物自動車,1


In [52]:
fig = px.bar(goods,x='現金以外の主な被害品',y='件数')
fig.show()

## その他のチャート（上級編）

複数の変数でチャートを作ると、より深い分析ができる。

In [53]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            color='施錠関係',
            template='seaborn')
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [55]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            color='施錠関係',
            template='seaborn',
            barmode='group')
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

↑上のチャートの凡例ののカテゴリーをダブルクリックするとどうなる？

`barmode='group'`を足すと何がどのように変わる？

In [56]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            color='施錠関係',
            barmode='group', # group the categories,
            template='seaborn'
            )
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

# Python Lesson: Functions 関数

突然だが、ちょっとここでPythonのレッスンを投入！次のステップでFunction (関数)を使うので、まずは簡単に学ぼう。

関数とは、特定のタスクを実行するために組織化され<span style="font-size:2em">【再利用可能】</span>なコードのブロックです。例えば、繰り返し同じ作業をするぞ！と分かったら、同じコードブロックをコピペするよりかは関数を作った方が良い。


In [57]:
# example of a simple function
def hello():
    print('こんにちは😀')

In [58]:
# call the function
hello()

こんにちは😀


In [63]:
# example of a function with an argument

#文字列の中に変数を埋め込めるのにfを使う

def hello(name):
    print(f'✨こんにちは {name}✨!')

In [64]:
# call the function
hello('Misuzu')

✨こんにちは Misuzu✨!


In [65]:
def add(a,b):
    print(a+b)

In [66]:
add(100,87)

187


## チャレンジ問題

メジャーリーグの野球の試合でピッチャーの時速が画面で表示されるが、これは大体MPH (miles per hour)なので、日本人には分かりにくい。

<img src="https://raw.githubusercontent.com/yohman/24-1-Reitaku-GIS/229177aa00887ceccb2a6d0cf2b0cd1263298cc4/Weeks/Week08/images/mlb%20mpg.jpeg" width=600>

では、このMilesをKilometersに変える関数を書きましょう！計算は：

`1 miles = 1.60934 kilometers`

In [67]:
# create the function
def kilo(a):
    print(1.60934*a)


In [68]:
# call the function
kilo(104)

167.37136


# Geocoding

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/Geocoding_01.png" width=400>

住所だけではマップイングできません。座標が必要です。なので、住所から座標を特定するプロセスが必要である。このプロセスを<h1>【ジオコーティング】</h1>という。


ジオコーティングと言えば、色んな方法があります。現在、日本で無料でジオコーティングサービスを提供しているのが国土地理院のジオコーティングAPI。

試してみよう。このようにURLをブラウザーで記入するだけで座標が返ってくるサービスである。国土地理院さん、とても便利なサービス、ありがとうございます！

https://msearch.gsi.go.jp/address-search/AddressSearch?q=麗澤大学

では、このプロセスに従って、アドレスから座標を出力する Python 関数を作成します。

In [69]:
# 関数を作成
def geocode(address):

    # ジオコーティングURL
    url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

    # try/exceptでエラーをキャッチ
    # 成功の場合
    try:
        s_quote = urllib.parse.quote(address)
        response = requests.get(url + s_quote)
        if len(response.json())>0:
            return response.json()[0]["geometry"]["coordinates"] 
        else:
            return False
        
    # 失敗の場合
    except:
        return False

関数の使い方は簡単！

In [70]:
geocode('南柏')

[139.953476, 35.845783]

In [74]:
geocode('我孫子１')

[140.012131, 35.873779]

In [73]:
geocode('ひがし野')

[139.995178, 35.95369]

## データフレームの準備

`kashiwa`のデータフレームの各行の住所をジオコーティングする前に`住所`と座標（`lat`,`lon`）のフィールドを作りましょう。

In [75]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 132 to 194
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             63 non-null     object
 1   手口             63 non-null     object
 2   管轄警察署（発生地）     63 non-null     object
 3   管轄交番・駐在所（発生地）  63 non-null     object
 4   市区町村コード（発生地）   63 non-null     int64 
 5   都道府県（発生地）      63 non-null     object
 6   市区町村（発生地）      63 non-null     object
 7   町丁目（発生地）       63 non-null     object
 8   発生年月日（始期）      63 non-null     object
 9   発生時（始期）        63 non-null     object
 10  発生場所           63 non-null     object
 11  発生場所の詳細        63 non-null     object
 12  施錠関係           63 non-null     object
 13  盗難防止装置の有無      63 non-null     object
 14  現金以外の主な被害品     63 non-null     object
dtypes: int64(1), object(14)
memory usage: 7.9+ KB


In [76]:
# 空の住所フィールドを作成
kashiwa['住所'] = ''

In [77]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 132 to 194
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             63 non-null     object
 1   手口             63 non-null     object
 2   管轄警察署（発生地）     63 non-null     object
 3   管轄交番・駐在所（発生地）  63 non-null     object
 4   市区町村コード（発生地）   63 non-null     int64 
 5   都道府県（発生地）      63 non-null     object
 6   市区町村（発生地）      63 non-null     object
 7   町丁目（発生地）       63 non-null     object
 8   発生年月日（始期）      63 non-null     object
 9   発生時（始期）        63 non-null     object
 10  発生場所           63 non-null     object
 11  発生場所の詳細        63 non-null     object
 12  施錠関係           63 non-null     object
 13  盗難防止装置の有無      63 non-null     object
 14  現金以外の主な被害品     63 non-null     object
 15  住所             63 non-null     object
dtypes: int64(1), object(15)
memory usage: 8.4+ KB


In [78]:
# 空だがfloatとしてフィールドを作成
kashiwa['lat'] = pd.Series(dtype=float)
kashiwa['lon'] = pd.Series(dtype=float)

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/concat.png" width=600>

↑で作った住所フィールドに次の 3 つのフィールドを連結します。

1. 都道府県（発生地）
1. 市区町村（発生地）
1. 町丁目（発生地）

In [79]:
# 住所フィールドを作成
kashiwa['住所'] = kashiwa['都道府県（発生地）']+kashiwa['市区町村（発生地）']+kashiwa['町丁目（発生地）']

In [80]:
# random sampleで上手く行ったかどうかをチェック
kashiwa[['都道府県（発生地）','市区町村（発生地）','町丁目（発生地）','住所']].sample(5)


,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,住所
187,千葉県,柏市,高田,千葉県柏市高田
134,千葉県,柏市,酒井根２丁目,千葉県柏市酒井根２丁目
188,千葉県,柏市,豊上町,千葉県柏市豊上町
172,千葉県,柏市,船戸,千葉県柏市船戸
133,千葉県,柏市,今谷上町,千葉県柏市今谷上町


これで準備は整えました。`kashiwa` のデータフレームを for loop に入れて住所を一つずつジオコーティングしよう。

この作業は数分かかるので注意。千件以上のデータフレームはなるべく避けよう。

In [81]:
# kashiwaデータフレームをループ
for i,row in kashiwa.iterrows():

    # ジオコーティング成功
    if geocode(row['住所']) != False:
        # 座標を変数に
        lon = geocode(row['住所'])[0]
        lat = geocode(row['住所'])[1]

        # データフレームに値をインプット        
        kashiwa.loc[i,'lon'] = lon
        kashiwa.loc[i,'lat'] = lat

        # 結果をprint
        print(row['住所'],lon,lat)
    
    # ジオコーティング失敗
    else:
        print(row['住所'],'ジオコーティング失敗')
        continue

千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市酒井根２丁目 139.967194 35.821594
千葉県柏市酒井根３丁目 139.965973 35.818859
千葉県柏市酒井根７丁目 139.95784 35.820915
千葉県柏市北柏１丁目 139.985413 35.874477
千葉県柏市北柏２丁目 139.989914 35.872643
千葉県柏市北柏２丁目 139.989914 35.872643
千葉県柏市北柏４丁目 139.986893 35.870586
千葉県柏市宿連寺 139.985229 35.887501
千葉県柏市宿連寺 139.985229 35.887501
千葉県柏市根戸 139.98645 35.88113
千葉県柏市根戸 139.98645 35.88113
千葉県柏市逆井 139.982468 35.815441
千葉県柏市加賀３丁目 139.97319 35.830082
千葉県柏市藤心 139.995834 35.820541
千葉県柏市増尾台４丁目 ジオコーティング失敗
千葉県柏市大井 140.00441 35.850155
千葉県柏市塚崎 139.996063 35.834927
千葉県柏市塚崎 139.996063 35.834927
千葉県柏市旭町６丁目 139.958267 35.857044
千葉県柏市篠籠田 139.943359 35.868122
千葉県柏市大室１丁目 139.95816 35.90826
千葉県柏市十余二 139.959305 35.882629
千葉県柏市松ケ崎 139.975952 35.879082
千葉県柏市松葉町６丁目 ジオコーティング失敗
千葉県柏市松葉町７丁目 139.963287 35.889534
千葉県柏市松葉町７丁目 139.963287 35.889534
千葉県柏市松葉町７丁目 139.963287 35.889534
千葉県柏市正連寺 139.950058 35.902401
千葉県柏市若柴 139.953995 35.891659
千葉県柏市若柴 139.953995 35.891659
千葉県柏市若柴 139.953995 35.891659
千葉県柏市若柴 1

In [82]:
# check your output, show address and lat/lon for 5 samples
kashiwa[['住所','lat','lon']].sample(5)


,住所,lat,lon
173,千葉県柏市船戸１丁目,35.914230,139.949768
168,千葉県柏市岩井,35.847050,140.034134
150,千葉県柏市塚崎,35.834927,139.996063
152,千葉県柏市旭町６丁目,35.857044,139.958267
162,千葉県柏市若柴,35.891659,139.953995


In [83]:
# geocode出来なかったデータ、latがNaNを削除
kashiwa = kashiwa[kashiwa['lat'].notna()]


## Let's map it!

待ってました、いよいよマップタイム！

In [84]:
# 全データの中央座標
center_lat = kashiwa.lat.mean()
center_lon = kashiwa.lon.mean() 

In [86]:
# make the map
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)
 
# kashiwaのデータフレームをループしてマーカーを作る
for index, row in kashiwa.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                  # tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m


## Heatmap

Foliumのエキストラ機能としてヒートマップが作れる！では、やって見よう。

まずはデータをヒートマップが必要とする形式を作る。

In [87]:
# make a list of lat/lon's
heatmap_lat_lon = kashiwa[['lat','lon']].values.tolist()

# view the list
heatmap_lat_lon

[[35.841019, 139.956406],
 [35.841019, 139.956406],
 [35.821594, 139.967194],
 [35.818859, 139.965973],
 [35.820915, 139.95784],
 [35.874477, 139.985413],
 [35.872643, 139.989914],
 [35.872643, 139.989914],
 [35.870586, 139.986893],
 [35.887501, 139.985229],
 [35.887501, 139.985229],
 [35.88113, 139.98645],
 [35.88113, 139.98645],
 [35.815441, 139.982468],
 [35.830082, 139.97319],
 [35.820541, 139.995834],
 [35.850155, 140.00441],
 [35.834927, 139.996063],
 [35.834927, 139.996063],
 [35.857044, 139.958267],
 [35.868122, 139.943359],
 [35.90826, 139.95816],
 [35.882629, 139.959305],
 [35.879082, 139.975952],
 [35.889534, 139.963287],
 [35.889534, 139.963287],
 [35.889534, 139.963287],
 [35.902401, 139.950058],
 [35.891659, 139.953995],
 [35.891659, 139.953995],
 [35.891659, 139.953995],
 [35.891659, 139.953995],
 [35.864445, 139.986511],
 [35.84705, 140.034134],
 [35.84705, 140.034134],
 [35.836792, 140.091583],
 [35.91423, 139.949768],
 [35.91423, 139.949768],
 [35.917824, 139.954544],

In [89]:
from folium.plugins import HeatMap

# make the map
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12,
               tiles='cartodbdark_matter')

# add the heatmap
HeatMap(
    data = heatmap_lat_lon,
    radius=15,    
).add_to(m)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)

# kashiwaのデータフレームをループしてマーカーを作る
for index, row in kashiwa.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                #   tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m

# 今日の課題

**DUE：日曜日の深夜12時まで！**

違う場所の犯罪マップ作りに挑戦。

このサイトから都道府県を選び、データをダウンロード：

https://www.npa.go.jp/toukei/seianki/hanzaiopendatalink.html

注意！ジオコーティングは時間がかかるので、データを千件以下に絞ってからジオコーティングをするように。